<br><br><br>
<h2><font color="#004D7F" size=6>Práctica 2</font></h2>



<h1><font color="#004D7F" size=5> SISTEMAS BASADOS EN REGLAS DIFUSAS DESCRIPTIVAS LINGÜÍSTICAS TIPO MANDANI. APRENDIZAJE DE SISTEMAS PARA CLASIFICACIÓN</font></h1>

<br>
<div style="text-align: right">
<font color="#004D7F" size=3>Guillermo Tomás Fernández Martin y José Miguel Puerta Callejón</font><br>
<font color="#004D7F" size=3>Guillermo.Fernandez@uclm.es</font><br>
<font color="#004D7F" size=3>Sistemas Basados en el Conocimiento</font><br>
<font color="#004D7F" size=3>Grado en Ingeniería Informática (UCLM)</font><br>
</div>

---

<a id="indice"></a>
<h2><font color="#004D7F" size=5>Índice</font></h2>

* [1. Introducción](#introduccion)
* [2. Metodología](#meto)
* [3. Implementación](#scikit)
* [4. Resultados](#c45)
* [5. Entrega](#entrega)


---

<br>

<a id="introduccion"></a>
# <font color="#004D7F"> 1. Introducción</font>

En esta práctica vamos a aprender y después utilizar para inferencia un sistema de reglas difusas para clasificación de tipo Mandani basado en variables difusas lingüísticas. Para ello vamos a partir, como siempre, de una base de datos con las variables predictoras y la variable objetivo a predecir. En el caso de esta práctica, la **lluvia en Australia**.

Para entender un poco mejor en que consiste un sistema de reglas difusas, vamos a ver dos ejemplos con otras bases de datos. Primero, un caso donde, a partir de la calidad de la comida y del servicio, decidiremos si dar propina o no. En base a esto, podemos aprender un sistema difuso con reglas de la forma:

<img src="https://drive.google.com/uc?export=view&id=1_s89dvmpugs2Ls-wa4WTY40DL8otf_ZS" width="500">

- Si _CC_ es Buena y _CS_ es Agradable entonces Propina es Si 
- Si _CC_ es Mala y _CS_ es Aceptable entonces Propina es No 
- Si _CC_ es Decente y _CS_ es Aceptable entonces Propina es Si 

<br>

Otro ejemplo, es el que ya conocemos de la calidad del vino: 

<img src="https://drive.google.com/uc?export=view&id=1AXL42lnta40u3XdgGuSAIrAm3ZqfYuKk" width="500">

- Si _NAl_ es Bajo y _NAc_ es Alto entonces Calidad es Mala 
- Si _NAl_ es Medio y _NAc_ es Bajo entonces Calidad es Buena 
- Si _NAl_ es Alto y _NAc_ es Bajo entonces Calidad es Buena 

El objetivo de esta práctica es el de obtener un sistema basado en reglas difusas para la predicción de la lluvia en Australia utilizando árboles de decisión y realizar un proceso de inferencia a partir de dicho sistema. Para ello, vamos a llevar a cabo 3 fases:
1. Realizar una transformación de las variables continuas a variables difusas.
2. Obtener un sistema de reglas basado en un árbol de decisión.
3. Implementar un sistema de inferencia para la obtención de predicciones.

Por último habrá que llevar a cabo una comparativa de los árboles obtenidos en la práctica anterior.



---

<br>

<a id="data"></a>
# <font color="#004D7F"> 2. Metodología</font>


En primer lugar, dada la base de datos hay que definir las etiquetas lingüísticas para cada variable continua de la base de datos, esto es, el número de etiquetas, su nombre semántico y su definición con los conjuntos difusos asociados a cada etiqueta, como, por ejemplo, en los casos anteriores, Nivel de Alcohol, cuyo universo del discurso corre entre 8º hasta 20º, 3 etiquetas, con nombres bajo, medio, alto y definiciones triangulares para medio y trapezoides para bajo y alto.

Para el resto de las variables discretas de nuestra base de datos no hace falta nada más que extraer el número de valores únicos que aparecen en las bases de datos para almacenar y definir sus posibles valores de etiquetas, esto es, si tenemos una variable tipo de vino, pues tendrá solo dos valores: blanco o tinto (si acaso rosado también), con lo que tendremos una variable discreta que pueda tomar esos valores de etiquetas.

Una vez definida esta parte de la base de conocimiento del problema en cuestión, tenemos que pasar a aprender las diferentes reglas de clasificación, para ello procederemos como sigue.

### <font color="#004D7F"> 2.1 Fuzzificación de la Base de Datos</font>

Este paso consiste en transformar los valores numéricos de la base de datos a su etiqueta difusa correspondiente. Para ello, lo que realizaremos es obtener el grado de pertenencia para cada etiqueta definida para la variable y valor en cuestión y quedarnos con la etiqueta con mayor grado de pertenencia. Una vez identificada la etiqueta con mayor grado de pertenencia pasamos a substituir el valor numérico por su etiqueta.




In [35]:
import pandas as pd
import copy

In [36]:
pd.DataFrame([[18.5, 7.5, 'Blanco', 'Buena'],[12,12.5,'Tinto','Mala']], columns=['Nivel de Alcohol', 'Nivel de Acidez', 'Tipo', 'Calidad'])

,Nivel de Alcohol,Nivel de Acidez,Tipo,Calidad
0,18.5,7.5,Blanco,Buena
1,12.0,12.5,Tinto,Mala


Se transformaría a la siguiente base de datos:

In [37]:
pd.DataFrame([['Alto', 'Bajo', 'Blanco', 'Buena'],['Bajo','Alto','Tinto','Mala']], columns=['Nivel de Alcohol', 'Nivel de Acidez', 'Tipo', 'Calidad'])

,Nivel de Alcohol,Nivel de Acidez,Tipo,Calidad
0,Alto,Bajo,Blanco,Buena
1,Bajo,Alto,Tinto,Mala


### <font color="#004D7F"> 2.2 Aprendizaje de un Árbol de Decisión</font>

Una vez que tenemos fuzzificada la base de datos podemos interpretarla como una base de datos discreta completamente y, por tanto, podemos utilizarla como entrada para aprender un árbol de decisión con los algoritmos que hemos desarrollado en la práctica anterior.

Esto es, podemos aprender un árbol a partir de la base de datos anterior pasada al valor de etiqueta de cada variable y valor. En el caso anterior podríamos tener un árbol de salida tal como el siguiente:


<img src="https://drive.google.com/uc?export=view&id=1gbA3pg0nvVbQ5YzWKWaZr0s-g6icJ6QM" width="500">

Con este resultado podemos tener las siguientes reglas:
- $r_1$: Si _NAl_ es Bajo entonces Calidad es Mala
- $r_2$: Si _NAl_ es Medio y _NAc_ es Bajo y Tipo es Blanco entonces Calidad es Buena
- $r_3$: Si _NAl_ es Medio y _NAc_ es Bajo y Tipo es Tinto entonces Calidad es Mala
- $r_4$: Si _NAl_ es Medio y _NAc_ es Alto entonces Calidad es Mala
- $r_5$: Si _NAl_ es Alto entonces Calidad es Buena

Fijémonos que las variables discretas en la base de datos original se pueden incorporar en el árbol de forma natural y por tanto en las reglas de salida pueden tener reglas con variables difusas y variables discretas sin ningún problema. En el ejemplo, tenemos la variable Tipo de Vino que es discreta por naturaleza.



### <font color="#004D7F"> 2.3 Inferencia en la clasificación de una instancia</font>

Una vez definido nuestro sistema de reglas difusas lingüísticas para clasificación extraídas del árbol de decisión, tenemos que ver cómo realizar la inferencia para clasificar una nueva instancia.

Esta inferencia se puede realizar directamente en el procesado del árbol, o también utilizando el sistema de reglas extraído. Las dos formas son equivalentes y podrás implementar la que quieras.

#### **Inferencia utilizando el Sistema de Reglas Difusas:**
En este caso se disparan todas las reglas con el ejemplo que tengamos, y calculamos el grado de cobertura que tiene cada una de las reglas. Es decir, multiplicamos el grado de pertenencia de la etiqueta correspondiente a la premisa de la regla para cada premisa. En el caso de tener una premisa discreta, no difusa, se multiplicaría por cero si no corresponden los valores de la premisa con el del ejemplo, o uno si corresponde, es decir es una premisa no difusa, con valores de verdad uno o cero.

La decisión final sobre la salida de la variable clase se realizará de la siguiente forma: cada regla tendrá un grado de cobertura o recubrimiento y un valor de salida para la variable clase, lo notaremos como $r_i:\{C_i,w_i\}$, donde $C_i$ es el valor de la clase y $w_i$ es el grado de cubrimiento de la regla al ejemplo. 

El valor de clase de salida para el ejemplo se calculará cómo la clase mayoritaria ponderada por su valor de recubrimiento o cobertura, es decir, para la clase $C_i$ el número de votos será $∑_{r_j/C_j==C_i}w_j$ , es decir sumamos por cada regla que tenga como salida $C_i$ el valor de su grado de cobertura. Una vez calculado esto para cada etiqueta de salida, nos quedamos con la mayoritaria.

Por ejemplo, imaginad que tenemos la instancia a clasificar: **Nivel de Alcohol = 15, Nivel de Acidez = 5, Tipo = Blanco**. Esperamos ver la Calidad del vino como salida.

- $r_1$: Si NAl es Bajo entonces Calidad es Mala —> $r_1:\{Mala,w_i=0\}; μ_{Bajo}^{NAl} (15)=0$
- $r_5$: Si NAl es Alto entonces Calidad es Buena —> $r_5:\{Buena,w_i=0.33\}; μ_{Alto}^{NAl} (15)=0.33$
- $r_4$: Si NAl es Medio y NAc es Alto entonces Calidad es Mala —> $r_4:\{Mala,w_i=0.22\}; μ_{Medio}^{NAl} (15)=0.66 * μ_{Alto}^{NAc} (5)=0.33 = 0,22$
- $r_3$: Si NAl es Medio y NAc es Bajo y Tipo es Tinto entonces Calidad es Mala —> $r_3:\{Mala,w_i=0\}$. Tiene peso 0 porque Tipo es Blanco y en la regla tenemos Tipo es Tinto.
- $r_2$: Si NAl es Medio y NAc es Bajo y Tipo es Blanco entonces Calidad es Buena —> $r_2:\{Buena,w_i=0.44\}; μ_{Medio}^{NAl} (15)=0.66 * μ_{Bajo}^{NAc} (5)=0.66 * 1 = 0,44$

En este caso tenemos la suma de:
- $Buena = r_2 (0.44) + r_5 (0.33) = 0.77$
- $Mala = r_1(0) +r_4(0.22) +r_3(0) = 0.22$

por lo que se clasificaría como Buena


---

<br>

<a id="data"></a>
# <font color="#004D7F"> 3. Implementación</font>

Para esta práctica el alumno debe llevar a cabo la implementación de las tres etapas descritas en la metodología, con el objetivo final de crear un sistema de reglas difusas para el problema de la predicción de la lluvia en Australia y ser capaz de realizar predicciones a partir de la inferencia. Se utilizará el paquete `scikit-fuzzy`, que podéis instalar con `pip install scikit-fuzzy`.

Primero hay que realizar el proceso de Fuzzificación de la base de datos de la lluvia en Australia, donde las variables continuas deben ser convertidas a etiquetas. Después hay que llevar a cabo el proceso de aprendizaje del árbol utilizando la implementación realizada en la práctica 1. Por último, a partir de este árbol, el alumno debe de implementar un método para llevar a cabo la inferencia a partir de la base de datos original.

Vamos a describir un poco más en detalle cada uno de estos pasos:



### <font color="#004D7F"> 3.1 Variables Difusas</font>

El primer paso será generar las variables difusas a partir de las variables de la base de datos. Para ello, se proporciona el esqueleto de una clase que os ayudará a estructurar las variables con los elementos necesarios para la transformación posterior de la base de datos. Para simplificar, se utilizarán conjuntos difusos triangulares.

Los atributos de la clase serán los siguientes:
- `universe`: El universo del discurso de la variable. **IMPORTANTE** `scikit-fuzzy` asume que el universo del discurso está ordenado para todas sus operaciones. Sin embargo, cuando extraemos una columna de la base de datos para generar la variable difusa no tiene por qué estar ordenada. Habrá que ordenarla para poder utilizar las funciones de `scikit-fuzzy` pero sin modificar la base de datos original.
- `memb_func`: Un diccionario con los grados de pertenencia de cada elemento del universo del discurso para cada uno de los conjuntos difusos que se generen de la variable difusa
- `labels`: Etiquetas de cada uno de los conjuntos difusos
- `name`: Nombre de la variable difusa

Las funciones de la clase serán las siguientes:
- `_membership_functions`: Función interna para generar el diccionario con las distintas funciones de pertenencia. A COMPLETAR
- `fuzzify`: Función que devuelve el mejor conjunto difuso para un valor dentro del universo del discurso. A COMPLETAR
- `plot`: Función que muestra de manera gráfica los conjuntos difusos de la variable. Ya viene dada.

Para utilizar la clase, habrá que calcular previamente los triángulos de los conjuntos difusos. Para ello, se utilizará la estructura `fuzzy_points`, que es diccionario con la definición de los triángulos y forma `{label: [start, peak, end]}` para cada triángulo.

Para generar esta estructura `fuzzy_points` se recomienda calcular las particiones de las variables de manera automática, ya sea calculando por anchura o por frecuencia. `Pandas` tiene funciones que podéis buscar para discretizar una variable tanto por anchura como por frecuencia, se recomienda utilizarlas.

In [38]:
import numpy as np
import skfuzzy as fuzz
import matplotlib.pyplot as plt
import scipy.interpolate as inter

class FuzzyVariable:
    def __init__(self, name, variable, fuzzy_points):
        new_var = variable.copy()
        self.universe = (new_var.sort_values(new_var.columns[0]))[new_var.columns[0]].unique() #Comprobar que es asi
        self.labels = fuzzy_points.keys()
        self.memb_func = self._membership_functions(variable, fuzzy_points)
        self.name = name
    
    def _membership_functions(self, variable, fuzzy_points):
        fuzz_dict = {}
        
        for label in self.labels:
            fuzz_dict[label] = fuzz.trimf(self.universe,fuzzy_points[label])
        return fuzz_dict
    
    def fuzzify(self, value):
        best = ''
        max_val = -1
        for label in self.labels: # Iterar por cada variable para ver la mejor
            # TODO: DEVOLVER EL CONJUNTO DIFUSO QUE MAYOR GRADO DE PERTENENCIA TENGA
            #interp_membership(x, xmf, xx)
            if fuzz.interp_membership(self.universe,self.memb_func[label], value) > max_val:
                max_val = fuzz.interp_membership(self.universe,self.memb_func[label], value)
                best = label
        return best

    def plot(self):
        fig, ax0 = plt.subplots(nrows=1)
        for label in self.labels:
            ax0.plot(self.universe, self.memb_func[label], linewidth=1.5, label=label)
        ax0.set_title(self.name)
        ax0.legend()
        plt.tight_layout()

#### La siguiente celda es una pequeña base de datos creada para probar el fuzzify.

In [39]:
bd = pd.DataFrame([[18.5, 7.5, 'Blanco', 'Buena'],[12,12.5,'Tinto','Mala'],[12,12.5,'Tinto','Mala'],[10,12.5,'Tinto','Mala'],[15,12.5,'Tinto','Mala'],[20,12.5,'Tinto','Mala']], columns=['Nivel de Alcohol', 'Nivel de Acidez', 'Tipo', 'Calidad'])
pd.DataFrame(bd['Nivel de Alcohol']).columns[0]
dic = {}
dic['bajo'] = [0,5,10]
dic['medio']= [10,15,20]
dic['alto'] = [20,25,30]
fuz_var = FuzzyVariable('Alcohol',pd.DataFrame(bd['Nivel de Alcohol']),dic)
fuz_var.fuzzify(13)



'medio'

### <font color="#004D7F"> 3.2 Transformador Fuzzy</font>

Para llevar a cabo la Fuzzificación de la base de datos, el alumno implementará un Transformador que se encargue de generar las variables difusas a partir de las variables continuas. La implementación consiste en generar las variables difusas necesarias y utilizar la función `fuzzify` para fuzzificar los valores. Se recomienda estudiar la función `DataFrame.apply` para realizar esta transformación

Los atributos de la clase son los siguientes:
- `sets`: Número de conjuntos difusos para cada variable, por defecto 3
- `labels`: Etiquetas difusas para los conjuntos. **DEBE TENER LA MISMA LONGITUD QUE `sets`**

La clase se dividirá en las funciones:
- `fit`: encargado de aprender las variables difusas a partir de los datos
- `transform`: creará una nueva base de datos sustituyendo los valores continuos por las etiquetas de las variables
- `fit_transform`: auxiliar para hacer los dos procedimientos en un solo paso.
- `_equal_width`: Esta función coge una variable en formato `NumPy` y genera cortes de igual anchura. Es decir, si la variable es `[1,2,3,4,5,6,7]` y tiene que hacer 3 cortes, devuelve los puntos `[1,4,7]`
- `_expand_points`: Esta función coge los puntos de `equal_width` y los expande para generar los triángulos en el formato `[start, peak, end]`

In [40]:
from sklearn.base import BaseEstimator


# Ejemplo de la estructura básica de un algoritmo que hereda de BaseEstimator siguiendo la estructura de SciKit
class FuzzyEncoder(BaseEstimator):
    # constructor
    def __init__(self, sets=3, labels=['Low', 'Mid', 'High']):
        self.fuzzy_vars = {}
        self.sets = sets
        self.labels = labels

    # siguiendo las guias de scikit-learn, creamos las funciones fit para el entrenamiento
    def fit(self, X):
        # TODO: APRENDER LAS VARIABLES DIFUSAS A PARTIR DE LOS DATOS
        columnas = X.columns
        for columna in columnas:
            if type(X[columna][0]) != str: 
                
                equal_width_points = self._equal_width(X[columna].to_numpy(), self.sets)
                fuzzy_points = self._expand_points(equal_width_points, self.labels)
                fuzz_var = FuzzyVariable(columna,pd.DataFrame(X[columna]),fuzzy_points)
                self.fuzzy_vars[columna] = fuzz_var

        return self
        
    # y transform para aplicar lo aprendido y transformar la base de datos creando una nueva
    def transform(self, X):
        new_X = X.copy()
        # TODO: MODIFICAR new_X CON LAS VARIABLES DIFUSAS APRENDIDAS
        for key in self.fuzzy_vars.keys():
            new_X[key] = new_X[key].apply(self.fuzzy_vars[key].fuzzify)
        return new_X
    
    def fit_transform(self, X):
        return self.fit(X).transform(X)

    def _equal_width(self, var, sets):
        try:
            cut = pd.cut(var, sets - 1)
            sol = [cat.left for cat in cut.categories] + [cut.categories[-1].right]
        except ValueError: 
            cut = pd.cut(var, sets - 1, duplicates='drop')
            sol = [var.min()] + [cat.left for cat in cut.categories] + [cut.categories[-1].right]
        sol[0] = var.min()
        return sol

    def _expand_points(self, fp, labels):
        ep = {}
        expanded_points = [fp[0]] + fp + [fp[-1]]

        for i in range(len(expanded_points) - 2):
            ep[labels[i]] = [expanded_points[i], expanded_points[i+1], expanded_points[i+2]]

        return ep

Ejemplo de uso de las funciones auxiliares: 

_Nota, utilizarlas solo dentro de la clase, esto es un ejemplo informativo_

Se ha de realizar el mismo preprocesamiento que en la práctica 1 con dos nuevas restricciones:
- La variable `Location` hay que eliminarla. Esto lo vamos a hacer para que el árbol que salga no ramifique demasiado
- Las variables de puntos cardinales hay que simplificarlas. Se han de quedar simplemente con los cuatro puntos cardinales principales (N, S, E, W)

In [41]:
# Preprocesamiento de la BBDD de la practica 1

df = pd.read_csv('weatherAUS.csv')
#Quitamos los valores nulos y las columnas Location y Date
del df['Location']
del df['Date']

for i in range (0,df.shape[1]):
    if type(df[(df.columns)[i]][0]) == str:
        df[(df.columns)[i]].fillna((df[(df.columns)[i]].mode())[0], inplace = True)
    else:
        df[(df.columns)[i]].fillna(df[(df.columns)[i]].mean(), inplace = True)

#Simplificamos los puntos cardinales
for elemento in df['WindGustDir'].unique():
    df.replace(elemento,elemento[0],inplace = True)



features = df.columns
attributes = features[:-1]
target = features[-1]
X = df[attributes]
y=df[target]
X


,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday
0,13.4,22.900000,0.6,5.468232,7.611178,W,44.00000,W,W,20.0,24.0,71.0,22.0,1007.7,1007.1,8.000000,4.50993,16.9,21.8,No
1,7.4,25.100000,0.0,5.468232,7.611178,W,44.00000,N,W,4.0,22.0,44.0,25.0,1010.6,1007.8,4.447461,4.50993,17.2,24.3,No
2,12.9,25.700000,0.0,5.468232,7.611178,W,46.00000,W,W,19.0,26.0,38.0,30.0,1007.6,1008.7,4.447461,2.00000,21.0,23.2,No
3,9.2,28.000000,0.0,5.468232,7.611178,N,24.00000,S,E,11.0,9.0,45.0,16.0,1017.6,1012.8,4.447461,4.50993,18.1,26.5,No
4,17.5,32.300000,1.0,5.468232,7.611178,W,41.00000,E,N,7.0,20.0,82.0,33.0,1010.8,1006.0,7.000000,8.00000,17.8,29.7,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145455,2.8,23.400000,0.0,5.468232,7.611178,E,31.00000,S,E,13.0,11.0,51.0,24.0,1024.6,1020.3,4.447461,4.50993,10.1,22.4,No
145456,3.6,25.300000,0.0,5.468232,7.611178,N,22.00000,S,N,13.0,9.0,56.0,21.0,1023.5,1019.1,4.447461,4.50993,10.9,24.5,No
145457,5.4,26.900000,0.0,5.468232,7.611178,N,37.00000,S,W,9.0,9.0,53.0,24.0,1021.0,1016.8,4.447461,4.50993,12.5,26.1,No
145458,7.8,27.000000,0.0,5.468232,7.611178,S,28.00000,S,N,13.0,7.0,51.0,24.0,1019.4,1016.5,3.000000,2.00000,15.1,26.0,No


In [42]:
fuzzy_encoder = FuzzyEncoder()
fuzzy_encoder.fit(X)

FuzzyEncoder()

In [44]:
fuzzy_X = X
fuzzy_X = fuzzy_encoder.transform(fuzzy_X)


In [45]:
fuzzy_X

,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday
0,Mid,Mid,Low,Low,Mid,W,Mid,W,W,Low,Mid,Mid,Low,Mid,Mid,High,Mid,Mid,Mid,No
1,Mid,Mid,Low,Low,Mid,W,Mid,N,W,Low,Mid,Mid,Low,Mid,Mid,Mid,Mid,Mid,Mid,No
2,Mid,Mid,Low,Low,Mid,W,Mid,W,W,Low,Mid,Mid,Mid,Mid,Mid,Mid,Low,Mid,Mid,No
3,Mid,Mid,Low,Low,Mid,N,Low,S,E,Low,Low,Mid,Low,Mid,Mid,Mid,Mid,Mid,Mid,No
4,Mid,Mid,Low,Low,Mid,W,Mid,E,N,Low,Low,High,Mid,Mid,Mid,High,High,Mid,Mid,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145455,Mid,Mid,Low,Low,Mid,E,Low,S,E,Low,Low,Mid,Low,Mid,Mid,Mid,Mid,Mid,Mid,No
145456,Mid,Mid,Low,Low,Mid,N,Low,S,N,Low,Low,Mid,Low,Mid,Mid,Mid,Mid,Mid,Mid,No
145457,Mid,Mid,Low,Low,Mid,N,Low,S,W,Low,Low,Mid,Low,Mid,Mid,Mid,Mid,Mid,Mid,No
145458,Mid,Mid,Low,Low,Mid,S,Low,S,N,Low,Low,Mid,Low,Mid,Mid,Mid,Low,Mid,Mid,No


In [51]:
y

0         No
1         No
2         No
3         No
4         No
          ..
145455    No
145456    No
145457    No
145458    No
145459    No
Name: RainTomorrow, Length: 145460, dtype: object

fuzzy_vars = FuzzyEncoder()
fuzzy_vars.fit(X)
fuzzy_X = fuzzy_vars.transform(X)

### <font color="#004D7F"> 3.3 Árbol C4.5</font>
En esta parte el alumno debe aprender un árbol utilizando la nueva base de datos fuzzificada y la implementación realizada en la práctica 1. Para ello, debe corregir los errores, si los hubiera, de la entrega anterior.


In [52]:
class Tree_c45:
    def __init__(self):
        # primero indicamos si el objeto es hoja o raiz
        self.is_leaf = True
        
        # atributos para el corte cuando el objeto es una raiz
        # indicamos el indice de la variable de corte y los valores
        self.var_index = -1
        self.cut_value = 0
        # los hijos deben ser objetos de Tree_c45
        self.list_of_childs = []
        
        # atributos cuando el Tree_c45 es una hoja
        # indicamos el valor de la clase y en class_count una tupla (casos con ese valor, casos totales en la hoja)
        self.class_value = -1
        self.class_count = 0
        
        # para el nivel de profundidad en el arbol de la raiz o la hoja
        self.level = -1
        
    def __str__(self):
        output = ''
        if(self.is_leaf):
            output += 'Class value: ' + str(self.class_value) + '\tCounts: ' + str(self.class_count)
        else:
            output += 'Feature '+ str(self.var_index)
            for i in range(len(self.list_of_childs)):
                output += '\n'+'\t'*(self.level+1)+str(self.cut_value[i])+': '+str(self.list_of_childs[i]) 
            
        return output
    
    # esta funcion nos servira para hacer predicciones y debe ser completada
    def predict(self,x):
        # TO DO
        #Crear dos arboles nuevos
        if (self.is_leaf):
            return self.class_value
        for child in self.list_of_childs:
            if x[self.var_index] == child.cut_value:
                return child.predict(x)

        return

In [65]:
from scipy.stats import entropy
from sklearn.base import BaseEstimator

class C45(BaseEstimator):
    # constructor con la profundidad del arbol, que por defecto es 2 y donde inicializamos el arbol que vamos a aprender
    def __init__(self, max_depth):
        
        self.max_depth = max_depth
        self.tree = Tree_c45()
        
    def __init__(self):
    
        self.max_depth = 2
        self.tree = Tree_c45()
    #siguiendo las guias de scikit-learn, creamos las funciones fit para el entrenamiento
    def fit(self, X, y):
        # esta será la llamada a la funcion recursiva que aprendera el arbol a partir de los datos
        self.make_cut(X, y, self.tree, 0)
        
    # y predict para las predicciones de casos nuevos
    def predict(self, X):
        t = []
        for i in range(len(X)):
            t.append(self.tree.predict(X.iloc[i]))

        return t
    
    # tambien se da una funcion score que calcula el accuracy
    def score(self, X, y):
        return np.sum(self.predict(X)==y)/y.shape[0]
    
    # esta es la funcion principal, encargada de encontrar la variable y su punto de corte que maximice la ganancia en base a la entropia.
    # se basa en que los cortes se van a realizar sobre variables continuas aunque la salida sea discreta
    def make_cut(self, X, y, current_tree, current_depth):
        

        # esqueleto con la funcionalidad basica
        
        # antes de realizar ninguna operacion comprobamos si hemos alcanzado el limite del arbol,
        # si es asi, actualizamos el arbol ya que estamos en un nodo hoja 
        # y no hace falta buscar variable ni continuar con las llamadas recursivas
        if current_depth >= self.max_depth:
            # TO DO

            #current_tree.is_leaf = ...
            current_tree.is_leaf = True
            #current_tree.class_value = ...


            if len(y.value_counts()) > 1:
                if y.value_counts()['No'] > y.value_counts()['Yes']:
                    current_tree.class_value = 'No'
                    current_tree.class_count = y.value_counts()['No']
                else: 
                    current_tree.class_value = 'Yes'
                    current_tree.class_count = y.value_counts()['Yes']
                return 
            else:
                auxiliar = y['RainTomorrow'].copy().to_numpy()
                current_tree.class_value = auxiliar[0]
                current_tree.class_count = y.value_counts()[0]


        # primero, obtenemos la entropia de la clase
        # TO DO        
        class_entropy = self.entropy(y)
        # segundo, deberemos recorrer todas las caracteristicas siguiendo estos pasos:
        # 1. Ordenar los valores de la caracteristica y aplicar a la clase
        
        #Declaramos las variables donde guardaremos los datos de por donde cortar el arbol
        finalinfogain = 0
        finalarraytablas = []
        finalarraytargets = []
        finalcol= None
        #Tenemos que mirar todas las columnas para saber cual es el mejor corte
        for col in X.columns: 
            #Creamos un dataframe auxiliar, t, en donde estarán los datos ordenados
            t = pd.concat([X,y], axis =1)
            # 2. Obtener los puntos de corte en base a la clase ordenada
           
            
            #En caso de ser variable catégorica (el tipo sea igual a string), declaramos variables auxiliares
            arraytablas = []
            arraytargets = []
            arrayentropia = []
            arrayprob = []
            #Y para cada valor unico de la columna, creamos una tabla y calculamos su entropía
            for label in X[col].unique():
                t1 = t[t[col] == label]
                X1 = t1[t1.columns[:-1]]
                
                #La tabla la guardamos para despúes, en caso de tener que cortar por aquí, poder pasarsela a la siguiente llamada de make_cut
                arraytablas.append(X1)
                    
                    
                y1 = t1[t1.columns[-1]]
                arraytargets.append(y1)
                    
                # 3.Calculamos la entropia y la probabilidad de ramificar por ahí
                e1 = self.entropy(y1)
                arrayentropia.append(e1)
                p1 = np.sum(y1.value_counts())/(np.sum(y.value_counts()))
                arrayprob.append(p1)
                
                # 4. Calculamos la ganancia con la entropia obtenida y la de la clase
            sum_entropy = 0
            for i in range(0,len(arrayprob)):
                sum_entropy += arrayprob.pop() * arrayentropia.pop()
            infogain = class_entropy - sum_entropy 
                
            # 5. Almacenamos la variable y el punto de corte con mayor ganancia, así como las tablas generadas y sus targets 
            if finalinfogain < infogain:
                finalinfogain = infogain
                finalcol = col
                finalarraytablas = arraytablas.copy()
                finalarraytargets= arraytargets.copy()
                
    
           
            # TO DO
            
            
        # antes de llamar a la funcion recursiva hay que actualizar los valores del arbol
        # TO DO
        #current_tree.is_leaf = ...
        current_tree.is_leaf = False
        #current_tree.level = ...
        current_tree.level = current_depth
        #current_tree.var_index = ...
        current_tree.var_index = finalcol
        #current_tree.cut_value = ...
        #El valor de corte va a ser distinto de un corte númerico (menor o igual y mayor), o corte categórico (cada valor único que tome la columna)
        
        if finalcol == None:
            return
        
        current_tree.cut_value = X[finalcol].unique()
        for i in X[finalcol].unique():
            #Y creamos un número de hijos igual al número de valores de corte
            current_tree.list_of_childs.append(Tree_c45())
                

        
        # para continuar con el algoritmo hay que hacer una llamada a la funcion
        # de forma recursiva para cada hijo
        for i in current_tree.list_of_childs:
            
            xi = pd.DataFrame(finalarraytablas.pop())
            yi = pd.DataFrame(finalarraytargets.pop())            
            
            self.make_cut(xi, yi, i, current_tree.level+1)
        return 0
    
    #Definimos la función entropía que se usa en make_cut
    def entropy(self,y):
        a = y.value_counts()/y.shape[0]
        entropy = np.sum(-a*np.log2(a))
        return(entropy)

El primer paso es entrenar el árbol con las nuevas variables fuzzificadas

In [79]:
t = C45()
t.fit(fuzzy_X, y)


Vamos a utilizar `scikit-fuzzy` para el sistema de control. Eso significa que se necesita un procesado previo para las variables discretas. Se necesitan dos elementos para crear un sistema basado en reglas con `scikit-fuzzy`:
- Universo del discurso. Para una variable discreta, será una enumeración de sus valores. Esto quiere decir que una variable `['N', 'S', 'E', 'W']` tendrá como universo del discurso `[0,1,2,3]`
- Una función de grados de pertenencia. En lugar de utilizar `fuzz.trapmf` o `fuzz.trimf`, la función será `1` cuando coincida con el elemento y `0` cuando no coincida. Para la variable anterior, 
  - `pert['N'] = [1,0,0,0]`,   
  - `pert['E'] = [0,0,1,0]`, 
  - etc.

Se recomienda crear un diccionario para cada una de las variables discretas para luego la generación de las reglas

In [80]:
def get_discrete(X, var):
    lusul = {}
    if type(X[var][0]) is str:
        lusul = {'universe': np.array(range(0,len(X[var].unique()))),
                'pert':{}
                }
        i = 0
        for label in X[var].unique():
            array = np.zeros(len(X[var].unique()),int)
            array[i] = 1
            lusul['pert'][label] = np.array(array)
            i = i+1    
    return lusul

Con este generador, dada la función `get_discrete` que nos genera las variables discretas, se consigue un diccionario por cada una de las variables discretas

#### Vamos a distinguir entre las variables discretas y las variables continuas.

In [81]:
columDiscretas = []
for var in X.columns:
    if type(X[var][0]) is str:
        columDiscretas.append(var)
discrete_variables = {att: get_discrete(fuzzy_X, att) for att in columDiscretas}

In [82]:
discrete_variables

{'WindGustDir': {'universe': array([0, 1, 2, 3]),
  'pert': {'W': array([1, 0, 0, 0]),
   'N': array([0, 1, 0, 0]),
   'S': array([0, 0, 1, 0]),
   'E': array([0, 0, 0, 1])}},
 'WindDir9am': {'universe': array([0, 1, 2, 3]),
  'pert': {'W': array([1, 0, 0, 0]),
   'N': array([0, 1, 0, 0]),
   'S': array([0, 0, 1, 0]),
   'E': array([0, 0, 0, 1])}},
 'WindDir3pm': {'universe': array([0, 1, 2, 3]),
  'pert': {'W': array([1, 0, 0, 0]),
   'E': array([0, 1, 0, 0]),
   'N': array([0, 0, 1, 0]),
   'S': array([0, 0, 0, 1])}},
 'RainToday': {'universe': array([0, 1]),
  'pert': {'No': array([1, 0]), 'Yes': array([0, 1])}}}

De igual manera, hay que crear un diccionario con la variable clase, que también es discreta. Se crea igual que el resto de variables discretas, pero se aconseja pasar esta variable por separado. 

Finalmente se crea el sistema basado en reglas difusas. Vamos a tener una clase abstracta general y una clase que extienda de esta y utilice el árbol de decisión para obtener las reglas.

La clase abstracta `RuleBasedSystem` se dividirá en las funciones:
- `fit`: Metodo abstracto que se programará en la clase que extienda de `RuleBasedSystem`
- `_create_rbs`: encargado de aprender el sistema basado en reglas. Tiene los siguientes parámetros:
  - `rule_list`: Lista de reglas con las que se creará el sistema de control
  - `fuzzy_variables`: Variables difusas extraidas de `FuzzyEncoder.fuzzy_vars`
  - `discrete_variables`: Diccionario con las variables discretas.
  - `target_variable`: Diccionario con la variable objetivo.
- `predict`: Devuelve la clasificación para una instancia de la base de datos segun el sistema basado en reglas
- `score`: Devuelve el score de la base de datos

In [84]:
from skfuzzy import control as ctrl
from abc import ABC, abstractmethod

class RuleBasedSystem(ABC):
    def __init__(self):
        self.ctrl = None

    def _create_rbs(self, rule_list, fuzzy_variables, discrete_variables, target_variable):
        # TODO: PASO 1
        
        # Hay que obtener todos los antecedentes. Un antecedente en 
        # scikit-fuzzy se crea con ante = ctrl.Antecedent(universe, variable_name)
        # Para ello, tanto en fuzzy_variables como en discrete_variables
        # deberia estar ya representado el universo de cada variable

        # Ademas, para cada antecedente hay que crear todos los conjuntos difusos
        # (o valores para las variables discretas) correspondientes. Tenemos las etiquetas
        # en las fuzzy_variables, y se pueden obtener tambien de las variables discretas
        # Para cada conjunto difuso se crea con ante[label] = pert, donde ya tenemos
        # las funciones de pertenencia tanto en fuzzy_variables como en discrete_variables
        
        self.target = target_variable

        dict = {}
        for var in fuzzy_variables:
            dict[var] = ctrl.Antecedent(fuzzy_variables[var].universe,var)
            for label in fuzzy_variables[var].labels:
                dict[var][label] = fuzzy_variables[var].memb_func[label]
        
        for var in discrete_variables:
            dict[var] = ctrl.Antecedent(discrete_variables[var]['universe'],var)
            for label in discrete_variables[var]['pert'].keys():
                dict[var][label] = discrete_variables[var]['pert'][label]


        

        #rule_list = [rule1,rule2,rule3]
        #rule1 = (antecedentes, consecuente)
        #antecedentes = [ante1,ante2,ante3]
        #ante1 = [nombre,etiqueta]



        # TODO: PASO 2

        dict['target'] = ctrl.Consequent(target_variable['universe'],'RainTomorrow')
        for label in target_variable['pert'].keys():
            dict['target'][label] = target_variable['pert'][label]



        # Para crear el consecuente se hace con conseq = ctrl.Consequent(universe, conseq_name)
        # Por eso es mejor pasarlo por separado. De igual manera, hay que asignar cada uno 
        # de los valores que puede tomar y sus funciones de pertenencia






        rules = []

        # TODO: PASO 3

        # Finalmente hay que generar las reglas. Para ello se crean con 
        # rule = ctrl.Rule(antecedent=antecedents, consequent=consequent)
        # Aqui trabajamos con los antecedentes y el consecuente que hemos generado
        # anteriormente. Con la lista de reglas que tenemos del arbol, generamos
        # las reglas con el formato de scikit-fuzzy. antecedents es una union logica 
        # de terminos, esto quiere decir, ante1[label1] & ante2[label2]...
        # De igual manera, el consecuente es conseq[label] para el valor correspondiente
        # de la regla. Habra que haber almacenado los antecedentes para poder acceder
        # a ellos en funcion del nombre de la columna que aparece en la regla
        i = 0
        for rule in rule_list:
            
            antes = dict[rule[0][0][0]][rule[0][0][1]] #Inicializamos "antes" con la primera instancia
            for antece in rule[0]:

                if str(antece[0]+"["+antece[1]+"]") == str(antes):
                    pass
                else:
                    antes = antes & dict[antece[0]][antece[1]]
            print(antes)
            print (dict['target'][rule[1]])
                
            rules.append(ctrl.Rule(antecedent = antes, consequent = dict['target'][rule[1]]))  
        
        
        
        
        
        self.ctrl = ctrl.ControlSystem(rules=rules)

        print(self.ctrl.rules)

    @abstractmethod
    def fit():
        pass
    
    def predict(self, x):
        sim = ctrl.ControlSystemSimulation(self.ctrl, flush_after_run=1000)
        # Para predecir, hay que añadir los valores de la instancia al sistema
        # de simulacion de control. Primero hay que ver si existe el antecedente 
        # en el sistema de control (posiblemente las reglas extraidas del arbol
        # no tengan todas las caracteristicas de la instancia). En caso afirmativo,
        # la variable 'temperatura' = 8.7 se añade como sim.input['temperatura'] = 8.7    
        for col in x.columns:
            for rule in self.ctrl.rules.all_rules:
                for ante in rule.antecedent_terms:
                    if str(col) in str(ante):
                        if type(x[col].unique()[0]) is str:
                            sim.input[col] = sum(discrete_variables[col]['pert'][x[col].unique()[0]] * discrete_variables[col]['universe'])
                        else:
                            sim.input[col] = x[col].unique()[0]
                        

        # TODO: IMPLEMENTAR

        
        # Despues se calculan los grados de las reglas
        sim.compute()

        # Finalmente se devuelve el resultado que se encuentra en sim.output['target']
        # si la variable a predecir se llama 'target'.
        
        # TODO: IMPLEMENTAR

        return sim.output['RainTomorrow']
    

    def score(self, X, y):
        all_predictions = np.array([round(self.predict(x)) for x in X])
        return np.sum(all_predictions==y)/y.shape[0]





La clase `TreeRuleBasedSystem` se dividirá en las funciones:
- `_extract_rules`: encargado de procesar el arbol ya entrenado para extraer las reglas. Se proporciona un formato sugerido para extraer las reglas pero se puede utilizar el formato preferido.
- `fit`: Metodo que entrena el sistema basado en reglas

In [73]:
class TreeRuleBasedSystem(RuleBasedSystem):
    def __init__(self):
        super().__init__()

    # Se recomienda este formato para extraer las reglas
    # del arbol, ya que simplifica la manera de tratar los 
    # antecedentes y consecuentes a la hora de generar el 
    # sistema basado en reglas. A continuación se da 
    # una función con el formr el formato,a que modificar para
    # que extraiga adecuadamente las reglas
    
    def _extract_rules(self, tree):
        # Un antecedente es una tupla (caracteristica, valor)
        
        # Ejemplo -> antecedente = ('WindDir3pm', 'W')
        
        # Una regla es una tupla ([antecedentes], consecuente)
        # Ejemplo -> rule = (antecedentes, consequent)
        
        #Hacemos una búsqueda en profundidad por el arbol para crear los antecedentes y cuando se llega al nodo hoja, creamos la regla.
        all_rules = []
        pila = []
        pila.append((tree,[]))
        while(len(pila) > 0):
            n = 0
            arbol = pila.pop()
            if arbol[0].is_leaf:
                all_rules.append((arbol[1],arbol[0].class_value))
            else:
                for label in arbol[0].cut_value: 
                    anteNuevo = copy.copy(arbol[1])
                    anteNuevo.append((arbol[0].var_index,label))
                    pila.append((arbol[0].list_of_childs[n],anteNuevo))
                    n += 1
        return all_rules


    def fit(self, tree, fuzzy_variables,discrete_variables, target_variable):
        rule_list = self._extract_rules(tree)
        self._create_rbs(rule_list, fuzzy_variables, discrete_variables, target_variable)

In [85]:
#La variable target se considera una variable discreta.
discrete_target = get_discrete(pd.DataFrame(y),"RainTomorrow")

In [103]:
discrete_target

{'universe': array([0, 1]),
 'pert': {'No': array([1, 0]), 'Yes': array([0, 1])}}

In [104]:
arbol = TreeRuleBasedSystem()
arbol.fit(t.tree,fuzzy_encoder.fuzzy_vars,discrete_variables,discrete_target)

Humidity3pm[High] AND WindDir3pm[S]
RainTomorrow[No]
Humidity3pm[High] AND WindDir3pm[N]
RainTomorrow[No]
Humidity3pm[High] AND WindDir3pm[E]
RainTomorrow[No]
Humidity3pm[High] AND WindDir3pm[W]
RainTomorrow[No]
Humidity3pm[Mid] AND RainToday[Yes]
RainTomorrow[No]
Humidity3pm[Mid] AND RainToday[No]
RainTomorrow[No]
Humidity3pm[Low] AND RainToday[No]
RainTomorrow[Yes]
Humidity3pm[Low] AND RainToday[Yes]
RainTomorrow[Yes]


In [105]:
x = pd.DataFrame(X.loc[1000:1000])
x

,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday
1000,7.0,21.3,0.0,5.468232,7.611178,E,35.0,E,N,7.0,20.0,72.0,44.0,1019.9,1014.4,8.0,4.50993,13.3,20.5,No


In [107]:
arbol.predict(x)

0.3733333333333334

---

<br>

<a id="data"></a>
# <font color="#004D7F"> 4. Resultados</font>

Para finalizar esta práctica, el alumno deberá ejecutar el árbol con diferentes profundidades y comparar los resultados con las versiones discreta y continua de la práctica 1. Se recomienda probar por lo menos profundidad 2, 3 y 4. Además habrá que comparar el rendimiento del árbol tanto en score como en tiempos de ejecución con el rendimiento del sistema basado en reglas correspondiente.

En esta ocasión no habrá que llevar a cabo un estudio tan exhaustivo como en la práctica 1. Simplemente habrá que comparar aquellas variables más importantes y observar si en esta versión difusa el comportamiento es el mismo o difiere.

Opcionalmente, se pueden obtener otros resultados como el tiempo de ejecución o una comparativa del tamaño de los árboles, número de reglas, etc. 


---

<br>

<a id="data"></a>
# <font color="#004D7F"> 5. Entrega</font>

Al igual que en la práctica anterior, el alumno subirá a campus virtual un notebook con la implementación del sistema de reglas y del proceso de inferencia.

Para aprobar la práctica será necesario que la implementación del algoritmo sea correcta, por lo que la parte correspondiente al árbol de decisión de la práctica 1 también debe funcionar. Se tendrán en cuenta para la nota la eficiencia de la implementación, así como la limpieza y los comentarios en el código.

La parte de resultados será necesaria llevarla a cabo para optar a la nota máxima, mientras que la parte opcional será valorada positivamente.

La entrega de esta práctica tiene de plazo hasta el día 24 de Abril de 2022.